# Stock Price Prediction Transformer
> Transformer architecture utilized for learning and predicting stock price time series
> data, incorporating _Time2Vec_ vectorization of the time dimension.

#### Sources
1.  https://towardsdatascience.com/stock-predictions-with-state-of-the-art-transformer-and-time-embeddings-3a4485237de6

In [1]:
import sqlite3
import warnings

import numpy as np
import pandas as pd
import pendulum as pdl
import plotly as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

import marketml

warnings.filterwarnings("ignore")

[I 200713 01:58:54 __init__:15] Project directory: /Users/taras/Code/projects/MarketML


## Hyperparameters

In [2]:
batch_size = 16
sequence_length = 32
num_heads = 6

## Load Data

In [3]:
db_path = marketml.project_dir.joinpath("data/db/stocks.sqlite")
connection = sqlite3.connect(str(db_path))
df = pd.read_sql_query(
    'SELECT * FROM prices WHERE ticker="AAPL"',
    connection,
    index_col="date",
    parse_dates=True
)

df.drop(columns=["ticker", "adjusted_close", "dividend_amount", "split_coefficient"], inplace=True)
df["volume"].replace(to_replace=0, method="ffill", inplace=True)
df.sort_index(inplace=True)
df.head(10)

,open,high,low,close,volume
date,,,,,
2000-03-23,142.00,150.38,140.00,141.31,5022900
2000-03-24,142.44,143.94,135.50,138.69,3990300
2000-03-27,137.63,144.75,136.88,139.56,2492700
2000-03-28,137.25,142.00,137.13,139.13,1812200
2000-03-29,139.38,139.44,133.83,135.94,2141400
2000-03-30,133.56,137.69,125.44,125.75,3700000
2000-03-31,127.44,137.25,126.00,135.81,3612800
2000-04-03,135.50,139.50,129.44,133.31,2933600
2000-04-04,132.63,133.00,116.75,127.31,5895800


In [4]:
figure = plt.subplots.make_subplots(rows=2, cols=1)
figure.add_scatter(x=df.index, y=df["close"], connectgaps=True, row=1, col=1)
figure.add_scatter(x=df.index, y=df["volume"], connectgaps=True, row=2, col=1)
figure.update_layout(title="AAPL Close Price and Trading Volume")

AttributeError: module 'plotly' has no attribute 'subplots'

In [5]:
columns = df.columns
for column in columns:
    df[column] = df[column].pct_change()
df.dropna(how="any", axis=0, inplace=True)

In [6]:
price_columns = columns.to_list()
price_columns.pop(-1)
min_return = min(df[price_columns].min(axis=0))
max_return = max(df[price_columns].max(axis=0))
delta_return = max_return-min_return

min_volume = df["volume"].min(axis=0)
max_volume = df["volume"].max(axis=0)
delta_volume = max_volume-min_volume

In [7]:
for column in price_columns:
    df[column] = (df[column] - min_return) / delta_return
df["volume"] = (df["volume"] - min_volume) / delta_volume